In [14]:
import torch
from torch import nn
import torch.nn.functional as F
import math

In [6]:
class PositionalEncoder(nn.Module):
    def __init__(self,d_model,max_seq_len=80):
        super.__init__()
        self.d_model=d_model

        pe=torch.zeros(max_seq_len,d_model)
        for pos in range(max_seq_len):
            for i in range(0,d_model,2):
                pe[pos,i]=math.sin(pos/(10000**((2*i)/d_model)))
                pe[pos,i+1]=math.cos(pos/(1000**((2*(i+1))/d_model)))
        
        pe=pe.unsqueeze(0)
        self.register_buffer('pe',pe)
    def forward(self,x):
        x=x*math.sqrt(self.d_model)

        seq_len=x.size(1)
        x=x+Variable(self.pe[:,:seq_len],requires_grad=False).cuda()#????
        return x

In [18]:
i=torch.randn((1,2))
i

tensor([[1.8012, 0.3071]])

In [21]:
F.softmax(i,dim=1)

tensor([[0.8167, 0.1833]])

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,heads,d_model,dropout=0.1):
        super().__init__()

        self.d_model=d_model
        self.d_k=d_model//heads
        self.h=heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    def attention(q,k,v,d_k,mask=None,dropout=None):
        scores=torch.matmul(q,k.transpose(-2,-1))/math.sqrt(d_k)

        if mask is not None:
            mask=mask.unsqueeze(1)
            scores=scores.masked_fill(mask==0,-1e9)

        scores=F.softmax(scores,dim=-1)